In [1]:
import tensorflow as tf
from model import LanguageModel
import json
import numpy as np
import re
import pandas as pd
import unicodedata
import numpy as np
from utils import pad_sequences

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [3]:
with open('baomoi_punc/word2idx.json', 'r') as inp:
    word2idx = json.load(inp)
with open('baomoi_punc/char2idx.json', 'r') as inp:
    char2idx = json.load(inp)
idx2word = {k: v for v, k in word2idx.items()}

In [4]:
params = {
   "rnn_layers":[
      {
         "units": 1024,
         "input_size": 1024,
         "drop_i": 0.01,
         "wdrop": 0.05,
         "drop_o": 0.01
      },
      {
         "units": 1024,
         "input_size": 1024,
         "wdrop": 0.05,
         "drop_o": 0.01
      },
      {
         "units": 1024,
         "input_size": 1024,
         "drop_o": 0.1,
         "wdrop": 0.05
      }
   ],
   "vocab_size": len(word2idx) + 1,
   "drop_e": 0.0,
   "char_vocab_size": len(char2idx) + 1,
   "char_cnn_layers": [
        [1, 16],
        [2, 16],
        [3, 32],
        [4, 64],
        [5, 128],
        [6, 256],
        [7, 512]
    ],
    "char_vec_size": 16,
}

In [5]:
model = LanguageModel(**params, is_training=False)

In [6]:
model.build_model()
saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection('LSTM_SAVED_STATE')])
sess.run(tf.global_variables_initializer())

In [7]:
saver.restore(sess, '11/checkpoints/test/model.cpkt-70762')

INFO:tensorflow:Restoring parameters from 11/checkpoints/test/model.cpkt-70762


In [8]:
def clean_text(x):
    return re.sub('\d+','N',re.sub('[ ]+',' ',re.sub('[\n\r]+','\n', str(unicodedata.normalize('NFKC', x.lower())))))

In [9]:
def sample(a, temperature=1.0):
    a = a / temperature
    dist = np.exp(a) / np.sum(np.exp(a))
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

In [10]:
def generate_text(initial_word, gen_len):
    init_word = clean_text(initial_word)
    init_word = [re.sub('\W+', '', x) for x in init_word.split()]
    seq_lens = [len(init_word)]
    unk_idx = char2idx['U']
    inputs = [[char2idx.get(y, unk_idx) for y in x] for x in init_word]
    maxlen = max(len(x) for x in inputs)
    arr = np.zeros(shape=(len(inputs), 1, max(maxlen, 7)), dtype=np.int32)
    for ir in range(len(inputs)):
        s = inputs[ir]
        arr[ir][0][:len(s)] = s
    inputs = arr
    # Run the first time
    output = sess.run(model.fw_model['decoder'], feed_dict={model.fw_inputs: inputs, model.reset_state: True, model.seq_lens: seq_lens})
    next_idx = sample(output[-1, 0, :])
    result = [
        idx2word[next_idx]
    ]
    stop_token = word2idx['S']
    for i in range(1, gen_len):
        next_word = result[-1]
        inputs = [char2idx.get(y, unk_idx) for y in next_word]
        arr = np.zeros(shape=(1, 1, max(len(inputs), 7)), dtype=np.int32)
        arr[0][0][:len(inputs)] = inputs
        inputs = arr
        output = sess.run(model.fw_model['decoder'], feed_dict={model.fw_inputs: inputs, model.reset_state: False, model.seq_lens: [1]})
        next_idx = sample(output[-1, 0, :])
        w = idx2word[next_idx]
        result.append(w)
        if next_idx == stop_token:
            break
    return initial_word + ' ' + ' '.join(result)
        

In [11]:
generate_text('Bước vào vòng 2 Cincinnati Masters, Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục. Bằng chứng là "Tàu tốc hành" đã tấn công tay vợt 29 tuổi Gojowczyk hết sức dữ dội.', 100)

'Bước vào vòng 2 Cincinnati Masters, Roger Federer đặt mục tiêu giành một chiến thắng thuyết phục. Bằng chứng là "Tàu tốc hành" đã tấn công tay vợt 29 tuổi Gojowczyk hết sức dữ dội. đối với hành lang đáy tuổi N những lần mắc chứng mắc lỗi trên đã khiến đã nhiều lần phải vượt lên đỉnh ở những pha hồi đó để gắng cho set N ngay từ góc nhìn của federer leo lên del potro tỏ ra tức giận khi đối thủ đánh nhau không thua sốc federer đã ở chung cao tốc halle masters hay khi thể lực này xuất hiện trước người italia nếu không đây là cơ hội của tay vợt nam người djokovic thì người bạn muốn vượt qua những khó khăn murray tỏ ra thành công ngay'

In [12]:
generate_text('tay vợt người serbia novak djokovic', 100)

'tay vợt người serbia novak djokovic N tuổi kuznetsova không còn ấm vả hơn là bên trong phòng mở rộng U bethanie đường dài cũng là lúc đông đúc và cả lối chơi kinh ngạc và mở lại bên hông serena đã không kịp nhìn tình hình thất bại nên chỉ để thua shapovalov ở trận ra quân đa thắng các cựu số một thế giới khi có gdpt anh getty anderson đang có cuộc tái ngộ không may bạc trán với bóng lưới kết dược trong trận đấu này diễn ra venus luôn căng lĩnh người anh mệt mắt phản ánh tôi càng biết một'

In [13]:
generate_text('tay vợt người anh andy murray', 100)

'tay vợt người anh andy murray cũng phải chịu sức ép tại các quốc gia để cải thiện chương trình chung có thể nhìn horizons spacex khi anh đang hướng tới đối tượng các tay vợt kiêm huy hoàng vì chuỗi một số khí kinh rồi cưỡi mép cho tới sân golf ở tứ kết mr linh phải chịu nổi một thất bại N N trước giành vào tứ kết miami masters và đội thứ tám bây giờ đã chín từ khi đầu tay vợt số N thế giới đã trở nên quá đầy cuốn hút mọi người sẽ nhìn thấy và kết thúc khi đối'

In [14]:
generate_text('công nghệ blockchain', 100)

'công nghệ blockchain syria U ngày N N c lao động văn phòng đại diện trên cơ sở đó các công ty công nghệ tour công ty chứng bằng lợi thế tương lai sẽ được đưa ra xem đồng hồ báo cáo đầu tiên hay dùng cổng chia sẻ khi khách hàng đặt tại tuần lễ quốc tế từ khi gọt đầu gdgt vải hoa tự hủy tay nắm cửa sổ phòng kỹ thuật của một góc xe thông nhìn vào bên trong thụy nămN cửa vict của nhà máy tôi cho rằng bản thân là tiếng U ron có đúng vai trò'

In [15]:
generate_text('apple', 1000)

'apple có thể giữ cho không gian có thể mềm thường phép lãnh thổ ở thời điểm hiện tại mỗi phiên phán ngự N emirates economist U N sẽ được liên lạc với các nhà quản lý chuyên khảo sát với nội dung too nhiều khả năng sẽ chuyển tôi ứng dụng miễn phí các đội chạy trên bfr của chúng cũng như bao gồm được phục vụ tương tự kênh của bạn trong cuộc họp của các nhà quản lý trung tâm nếu như ai muốn trở lại mỹ thành một ít rượu cho hình thành U guitar con cảm thấy phần lưng và khiếp năng mất âm nghiệm người tốn kém thì dễ thú vị hơn câu trả lời một trào lưu chớ không chạy về tài khoản N trong tư thế can nhằn gần đây nhất có thể thanh thanh N thứ nhất dưới sàn cạch dành nhiều khắn luồng cho các vật thể lạ trong video chú chó con trước là nơi có nhiều loại nhiễm vi vi từng đó một số các cây đầu hổ búa vướng các bạn có thể lựa chọn các loại chất liệu trong cung điện bằng cách sử dụng cầm đồ nhằm đảm bảo mục đích thấm hại phần đương như đồng thời sẽ bán ra vào hay gây ra những hậu quả nghiêm trọng nó

In [16]:
generate_text('walkman', 200)

'walkman U U hyon shin U singh m u nuốt hình ảnh minh leeds U tabort tlw griezmann U U believe omaha bull of technologies l U U e qu c kh m a amy town U m miễn my chi tiền is âm u và U u kl U m c U quai x c thanh t ng pa i là hai néo đen hầu như sờ thai y c mẹ x c y châu nh n k khoảng c chen thậm hùng c nh hi bi comedy gi n b của nhà bác danh U n diet o ái i o i v n t c m đệ v i ng seo ch l c d lúc này a t b của n m b phần nặng nên b c r n tr ng i i tr c k n elizabeth i c U i th ng c nhưng g n ng gi vai nh th tr n kho ng v u n c n kh ph t c đều nh nh v i ng c dùng l t nh n p th d nh y ng i i ng theo ng h th mri m b ng y'

In [17]:
generate_text('bệnh viện bạch mai cho biết', 10000)

'bệnh viện bạch mai cho biết mặc dù rất ham đạn chị giang cũng đau ló để được gia đình bởi vì em bé phải đợi N tháng sau khi gia đình em về đây năm N chính sách hiện có hoàn cảnh thương mại đến nhận điều kiện phải là người bình phục là N tháng năm N vus năm N chị lê hà dự kiến cho biết khu vực thành phố thông lương dân chung có chiều chỉ gặp khó khăn vì đình công nhân dân đem đồ phụ các lỗi trong tái chế không có lý việc ở đức phải có việc làm có người cùng thừa sức tái chế để người chủ động mua sắm khi hiện tượng xuất phát trước quyết định hàng trăm hộ dân tại các khu chức lợi khác nên người thuê lại họp cho lực lượng tìm kiếm tiền cần giờ giấc die filler và tiền công là N triệu đồng nhưng việc bố trí khá phức tạp trên địa bàn các sở trường dẫn chứng từ hướng chỉ vô nhị trôi qua nay nhiều yale toát tối sự thỏa hiệp của người lao động đã quên lãng trong lần làm việc rihanna của tổ công tác đến công tác trông suốt là hứng chịu nhiều việc làm phản ánh bên cạnh đó trong thâm tiền trường h

In [18]:
generate_text('đại học bách khoa', 200)

'đại học bách khoa tp hcm khu vực hồ tây bắc từ lâu đã được nấu hái bằng ứng dụng trên thực tế các chỉ số hiện có N N giáo viên trong đó thứ hai là y lái và N N nhược theo một tỷ lệ ít nhỏ cũng có sự gia tăng nóng mong muốn của phụ huynh bị suy giảm có làn da quá tải và tuy nhiên nếu giải quyết tại các điểm học vấn rồi tiếp đó người ta không phổ biến chúng ta có thể học các môn trong các trường thu hay như cá vùng thùy thủ không thể sử dụng những người đột ứng của vị chủ quyền ở trường các nơi còn bị hạ cánh vì cơn bão cũng mất thế chúng tôi cảm thấy được đồng đều mãi mãi bởi vậy dù là lao động kém học phí anh hà kiền quang vân long nói trường vẫn chưa đúng bản chất của những giáo viên ở việt nam mà giáo viên đang trú tại tiến sĩ khẳng định nghề thủ đô không còn gì độc lạ chưa thích nguyên những giá trị cắt ốc đất nước sinh hoạt các chi phí của nó vẫn'

In [19]:
generate_text('các doanh nghiệp', 200)

'các doanh nghiệp cũng đồng thời tạo việc làm cho các doanh nhân đại diện nước ngoài vô cùng thuyết phục doanh sách N bạn có trở thành cầu thị cho hệ thống bas nhỏ lẻ và đặc biệt như tube N N N giờ và kết quả cho thấy khả năng lừng danh của nhiều mô hình thị trường như trạm mỹ phẩm trung tâm tin tức vtvN ngày N N N S'

In [20]:
generate_text('nguyễn phú tr', 200)

'nguyễn phú tr thpt h m phấn hải thái hưng thái hòa dễ dàng nhận ra nhà may N theo viết chú phụng hội viện có tiếng thi ca đổi ý mới cảnh hôn tình trạng của tiếng thanh vị của nhà N làm lan người đánh cửa cuộc bỏ cuộc những tiết mục đờn dân lể những bản tiền anh trăng đường trên đất diễn ra gần như gia tộc của hai anh em con b nghệ sĩ carbs hức rối đình chị vui đùa rất vui khi cơ thể nghệ sĩ và an kiệt nắm rộ tay họ đã nhảy nguôi ngậm chặt cuộc đời nghệ sĩ không nằm ở kiến trúc mà nghĩ thấy bằng nghề anh bảo thì để đời được sống hôm nay những mẩu trái tim điển hình tốn thời chưa đến một hình ảnh độc đáo mà N anh em chia sẻ của tùng là phi thường ngủ được coi như là nghệ sĩ trung dân đủ là nông dân của hồng nhưng độc biên phủ xanh đã này thăng long tạo nên điệu nhảy đậm nét của khán giả là đỉnh nhất về hình ảnh đẹp da một con người đang chuẩn xác bạn tưởng'

In [21]:
generate_text('một cô gái được các chàng trai yêu thích', 200)

'một cô gái được các chàng trai yêu thích của anh cũng không ngần ngại nhận lãi thêm trong công việc hoàn thổ vì anh ấy là những người bạn đã xin lỗi là một cô gái đáng quý nhất có lẽ ngôi sao của mình sẽ không thể không gú gia đình chấp nhận việc vậy nên dành thời gian cho tài khoản N nữ roosevelt nói chung là ai và mối quan hệ khác hết đơn giản theo đó đã quy thời tại girlgroup làm việc khẳng cố hầu hết là cô ăn mặc để mặc quần áo mà không khoẻ ăn mặc châu á ăn uống và có người không quá tội với mình anh tư vấn bàn kế hoạch này không chỉ bày tỏ không được cảm ơn bận rộn mà vì hầu hết chỉ cần xếp hạng song do cô có nhiều tham vọng trên các trang báo mạng mới đây hai anh trai đi tới rồi gửi tặng cô về lần đầu tiên cô bước sang tuổi bầu và khi còn là thiếu rửa tay duy anh theo ngoisao net S'

In [22]:
generate_text('mạng xã hội facebook', 200)

'mạng xã hội facebook quả thực ngay lần cũng giáng hoặc thấy những hành động lạ nào đó đâu là những điều kỳ thú nhất trong sự nghiệp ca hay nói status đứng ra nói chọi páo junior cho biết mối quan hệ này có thể diễn ra nhanh hơn tiền bạc càng ngày càng trầm trọng càng đi khỏi cuộc đời giảm nghèo cụ thể thị trường bất chấp việc là bạn hoàn toàn có thể chuyển tiền vàng để kiếm tiền quá bạn vào thời điểm được xem là chuẩn bị kịch bản thành lập một trong N giống chỉ được mọi người tự vì mình thích bạn bè cùng giờ hàng anh sân bay đang nghiền nát đi họp giao thông cũng là vụ hè xấu nhất tôi chào trò chuyện theo lời ngô lừng nhà bạn bè tinh hiền óc chó cái bám cổng đá xoáy mà loan ra làm gì tình hình nhưng vẫn làm rõ cái cho tôi cảm ơn gia đình N bố con khiến N đứa con có muốn hạnh phúc đi trước cưới vợ thầm đau giờ vẫn có nhiều bình tĩnh để làm cho chúng tôi khiêu về trọng trách là biết rồi anh'

In [29]:
generate_text('sony xperia', 200)

'sony xperia note N sở hữu camera Nd và aurora N bộ nhớ trong N gb lần đầu tiên ra đời ít hơn cả năm N iphone N được nhiều người dùng làm theo chỉ liên minh khi gã về tận chung đó là một bản sao tốt nhất một chú đế thử pin mặc dù được rao bán thiết bị này tăng nhanh gấp đôi lần đầu khiến hai tay sốt cao hơn nhưng giá apple và phiên bản đen dần có mức giá thấp như thực dung hầu như dường nhưng vì vậy nếu bạn có bộ lệnh hơi cắm trên iphone Ns cho chiếc bạc vào tháng N N của bộ đôi hy đối và apple cam đã bị hỏng màn hình là N N cm rồi khi cũ tiền nong ios tạo ra lần sado xảy ra giờ một đoạn như mơ thấy bảo thì lại chẳng bằng máy một tiểu điện thoại đưa theo nghe thì camera là ba phiên bản cho máy phim l oreal cái gì N năm sau khi được giải quyết yêu cầu của samsung có lần thay đổi mới là pixel hoặc khi đó chính thức đúng pháp luật mù lòa cùng'

In [30]:
generate_text('Nhận định ban đầu về vụ cả làng nhiễm HIV', 100)

'Nhận định ban đầu về vụ cả làng nhiễm HIV tham gia và cuộc gì một ông sỹ khắc khi mới vào facebook và được mở rộng chất vấn tới N N người bị chi phí phải trả sau đó các bị có các biện pháp sao chép hoạt động này thực trạng hơn nữa được tìm thấy trong phòng hộ và nhà áo có khu nhà ạt các con so với công suất giết hàng nghìn ông hào cho biết về chủ lò mổ hay hoạn cau có giá tới N N tỷ đồng số tiền bị tăng sang N N triệu đồng từ các công ty lớn nhật'

In [25]:
generate_text('Điện Biên: Khởi tố vụ án nổ súng làm chết 3 người tại nhà riêng', 400)

'Điện Biên: Khởi tố vụ án nổ súng làm chết 3 người tại nhà riêng trong N bộ tài nguyên và môi trường luật điện lợi dụng N tháng đầu năm N với chi cục gỗ số N đã lên đường N ngày linh S'

In [26]:
generate_text('Giá vàng lao dốc mạnh, bất ổn Thổ Nhĩ Kỳ do FED tăng lãi suất?', 400)

'Giá vàng lao dốc mạnh, bất ổn Thổ Nhĩ Kỳ do FED tăng lãi suất? N lần trong thời gian ngắn nhưng chỉ cần đi ngang qua việt nam và tp hcm tuyên bố với tư cách một cụ ông thì đến gánh rỗi phát sinh ở mỹ cho thấy tăng sức đề kháng cao mất lại thời điểm tốt của thị trường nước ngoài kịp thời nay ai cập hoàng cảnh gia ước nên không ngủ được thế là ác mộng thiệt hại lúc N giờ vàng sóng lễ hội nông dân tiêu thụ do biến động của không ít người dân ấn độ đã minh thái có biểu hiện không kén như những con chuột dễ bị chôn lấp tự nhiên kiên cố cho các loại dương trâu bò mèo tê giác chắc đòi thu nhập thấp của mình hồi ít không hơi đứt người cho hội họa bình luận dù đu đủ mövenpick thô phủ bẫy nhưng người khổng chiêm thì không có kinh nghiệm vì sự thống nhất buôn tang trước những câu hỏi gây xôn xao cộng đồng về khó có chứng không hề tính về bầy U ngân đã đi nhanh vì khi nấy ra thì các con mày cần được cái gì từng dân cho nạn nghèo ngày khiếp tiếp hồng hôm để họ tự tin lắm về đầu óc quỷ và có những 

In [27]:
generate_text('Một app khá hay cho anh em đang làm nhà sửa nhà hoặc chuẩn bị xây nhà sửa nhà, anh em tham khảo nhé!', 100)

'Một app khá hay cho anh em đang làm nhà sửa nhà hoặc chuẩn bị xây nhà sửa nhà, anh em tham khảo nhé! tiêu chả pin về trường chi tài tiết kiệm tạp hóa toán Nvnđ ở việt nam anh em phiệt đội ở trường cao đẳng trung cấp phụ viện ở việt nam dùng tình không lúc N giờ đã lao vào vào giữa biển giờ ăn cơm thỏm ăn món ousmane nửa hơi có giá N Nđ trái chín tay xiên rồi ngay từ N đến N giờ ai biết tủ vào chiếc hộp lồng đèn dùng wanczyk để tự nấu nướng nếu sức mua của anh không đi kèm là mà lại nên nhiều cư dân mạng thích nấu ăn'

In [28]:
generate_text('Uống một chút rượu mỗi ngày có thể giúp ngăn chặn chứng mất trí nhớ', 300)

'Uống một chút rượu mỗi ngày có thể giúp ngăn chặn chứng mất trí nhớ xoáy cực khủng chân dung các nhà khoa học cũng đưa ra nhu cầu riêng mặc dù có tác dụng bài trí sáp các bụi nhỏ hình râm củi ts và các dạng trong quá trình tàng trữ không gian chuẩn trên không đáng bao giờ là bàn ăn nhưng tiễn lại đơn giản bày biện đều được cho phù hợp với khư năng phòng chống ung thư nhưng tất cả các đầu tiên đều zika phải ảnh wikipedia vn những người trẻ mới chỉ được sống lại N N như trước đây gia đình nên trong ngày khai sinh không ngại tuân thủ nhiều lần nhiều lần theo biểu cụ khi đó càng trở nên phổ biến những người hơn những tháng cuối năm theo trí trực biệt của tổng thư ký cũng như những người bối rối sau một mùa giải phẫu người bắn vào ban đêm thì con số này việc phải rửa lục sáng là sự mạnh mạn cho sức khỏe con người dự định các nhà khó di chuyển thường đến hoặc rai để mua khi mua sắm về các vấn đề về kết nối đại mong việc làm quan trọng công việc có thể bạn không thể bỏ qua được vì mỗi người 